In [2]:
import pandas as pd 

test = pd.read_csv('test.csv')

sample = pd.read_csv('sample_submission.csv')

train = pd.read_csv('train.csv')

# 전처리 

In [3]:
import numpy as np


# Number of Dependents
train['Number of Dependents'] = train['Number of Dependents'].astype('Int64')
train['Number of Dependents'] = train['Number of Dependents'].astype('category')
train['Number of Dependents'] = train['Number of Dependents'].cat.add_categories('Missing')
train['Number of Dependents'] = train['Number of Dependents'].fillna('Missing')


# Age
train['Age'] = train['Age'].fillna(train['Age'].median())


# Annual Income
train['Annual Income'] = train['Annual Income'].fillna(train['Annual Income'].median())


# Health Score
train['Health Score'] = train['Health Score'].fillna(train['Health Score'].median())


# Credit Score
train['Credit Score'] = train['Credit Score'].fillna('Missing')


# Customer Feedback
train['Customer Feedback'] = train['Customer Feedback'].fillna('No Feedback')


# Marital Status
train['Marital Status'] = train['Marital Status'].fillna('Unknown')


# Vehicle Age 결측 행 삭제
train = train[train['Vehicle Age'].notnull()]


# Insurance Duration 결측 행 삭제
train = train[train['Insurance Duration'].notnull()]

In [4]:
train.isnull().sum()

id                           0
Age                          0
Gender                       0
Annual Income                0
Marital Status               0
Number of Dependents         0
Education Level              0
Occupation              358074
Health Score                 0
Location                     0
Policy Type                  0
Previous Claims         364028
Vehicle Age                  0
Credit Score                 0
Insurance Duration           0
Policy Start Date            0
Customer Feedback            0
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

# train1 1-1 unknown으로 대체  2-1 0값으로 처리(수집 시 0을 null로 기입했을 가능성)

In [5]:
train1 = train.copy()
train1['Occupation'] = train1['Occupation'].fillna('Unknown')


train1['Previous Claims'] = train1['Previous Claims'].fillna(0)

In [6]:
train1.isnull().sum()

id                      0
Age                     0
Gender                  0
Annual Income           0
Marital Status          0
Number of Dependents    0
Education Level         0
Occupation              0
Health Score            0
Location                0
Policy Type             0
Previous Claims         0
Vehicle Age             0
Credit Score            0
Insurance Duration      0
Policy Start Date       0
Customer Feedback       0
Smoking Status          0
Exercise Frequency      0
Property Type           0
Premium Amount          0
dtype: int64

# train1 Catboost 적용


In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# 2. 타겟 설정
target1 = 'Premium Amount'
y1 = train1[target1]
X1 = train1.drop(columns=[target1])

# 3. 결측치 처리 및 범주형 컬럼 지정
cat_features1 = X1.select_dtypes(include=['object', 'category']).columns.tolist()

# NaN을 문자열 'Missing'으로 변환 (CatBoost 호환)
for col in cat_features1:
  X1[col] = X1[col].astype(str).fillna("Missing")

# 숫자형 컬럼 NaN은 중앙값으로 대체
for col in X1.select_dtypes(include=[np.number]).columns:
  if X1[col].isnull().sum() > 0:
      X1[col] = X1[col].fillna(X1[col].median())

# 4. 학습/검증 데이터 분리
X_train1, X_val1, y_train1, y_val1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

# 5. 모델 정의
model1 = CatBoostRegressor(
  iterations=1000,
  learning_rate=0.05,
  depth=6,
  eval_metric='MAE',
  cat_features=cat_features1,
  random_seed=42,
  verbose=100
)

# 6. 모델 학습
model1.fit(X_train1, y_train1, eval_set=(X_val1, y_val1), use_best_model=True)

# 예측 및 평가
y_pred1 = model1.predict(X_val1)
mae = mean_absolute_error(y_val1, y_pred1)
mse = mean_squared_error(y_val1, y_pred1)
rmse = np.sqrt(mse)
r2 = r2_score(y_val1, y_pred1)

print(f"train3 MAE:  {mae:.4f}")
print(f"train3 MSE:  {mse:.4f}")
print(f"train3 RMSE: {rmse:.4f}")
print(f"train3 R2:   {r2:.4f}")

0:	learn: 667.9864127	test: 668.7337461	best: 668.7337461 (0)	total: 1.11s	remaining: 18m 30s
100:	learn: 653.8314582	test: 654.4972113	best: 654.4972113 (100)	total: 1m 41s	remaining: 15m 1s
200:	learn: 650.5209692	test: 651.1096608	best: 651.1096608 (200)	total: 3m 13s	remaining: 12m 48s
300:	learn: 648.0365892	test: 648.6548364	best: 648.6548364 (300)	total: 4m 56s	remaining: 11m 27s
400:	learn: 646.5200491	test: 647.0460065	best: 647.0460065 (400)	total: 6m 45s	remaining: 10m 6s
500:	learn: 645.8414141	test: 646.4735110	best: 646.4735110 (500)	total: 8m 46s	remaining: 8m 44s
600:	learn: 645.3976426	test: 646.1379266	best: 646.1379266 (600)	total: 10m 33s	remaining: 7m
700:	learn: 645.0681170	test: 645.9437184	best: 645.9437184 (700)	total: 12m 31s	remaining: 5m 20s
800:	learn: 644.3945388	test: 645.3196472	best: 645.3196472 (800)	total: 14m 20s	remaining: 3m 33s
900:	learn: 644.0194156	test: 645.0804568	best: 645.0752155 (894)	total: 16m 5s	remaining: 1m 46s
999:	learn: 643.6440439

In [11]:
mean_premium = train1['Premium Amount'].mean()
mae = 644.7857
mape = mae / mean_premium * 100
print(f"Mean Premium: {mean_premium:.2f}")
print(f"Validation MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")


Mean Premium: 1102.54
Validation MAE: 644.79
MAPE: 58.48%


In [10]:
import pandas as pd
fi = pd.Series(model1.get_feature_importance(), index=X_train1.columns)
fi.sort_values(ascending=False).head(10)


NameError: name 'model1' is not defined

In [8]:
train1.dtypes


id                         int64
Age                      float64
Gender                    object
Annual Income            float64
Marital Status            object
Number of Dependents    category
Education Level           object
Occupation                object
Health Score             float64
Location                  object
Policy Type               object
Previous Claims          float64
Vehicle Age              float64
Credit Score              object
Insurance Duration       float64
Policy Start Date         object
Customer Feedback         object
Smoking Status            object
Exercise Frequency        object
Property Type             object
Premium Amount           float64
dtype: object

# 모델 개선도 

# 상관계수 시각화

In [ ]:
# import pandas as pd
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# import scipy.stats as stats
# from sklearn.feature_selection import mutual_info_regression
# from sklearn.preprocessing import LabelEncoder
# # 전처리: Number_of_Dependents_bin 생성
# train['Number of Dependents'] = train['Number of Dependents'].fillna(0)
# def bin_dependents(x):
#     if x == 0:
#         return '0'
#     elif x <= 2:
#         return '1-2'
#     else:
#         return '3+'
# train['Number_of_Dependents_bin'] = train['Number of Dependents'].apply(bin_dependents)
# # 1) 수치형 간 Pearson 상관분석
# num_cols = ['Age', 'Annual Income', 'Health Score', 'Vehicle Age', 'Insurance Duration']
# corr = train[num_cols].corr(method='pearson')
# plt.figure(figsize=(8, 6))
# sns.heatmap(corr, annot=True, cmap='coolwarm', square=True)
# plt.title(':막대_차트: Pearson Correlation: Numeric Features')
# plt.show()
# # 2) 범주형-수치형 분석: Boxplot + ANOVA
# sns.boxplot(x='Marital Status', y='Age', data=train)
# plt.title(':다트: Age by Marital Status')
# plt.show()
# grouped = [train.loc[train['Marital Status'] == g, 'Age'].dropna() for g in train['Marital Status'].unique()]
# anova_result = stats.f_oneway(*grouped)
# print('ANOVA p-value:', anova_result.pvalue)
# # 3) 범주형-범주형 분석: 카이제곱 + Cramér’s V + Countplot
# contingency = pd.crosstab(train['Marital Status'], train['Number_of_Dependents_bin'])
# chi2, p, dof, expected = stats.chi2_contingency(contingency)
# print('Chi2-test p-value:', p)
# def cramers_v(conf_matrix):
#     chi2 = stats.chi2_contingency(conf_matrix)[0]
#     n = conf_matrix.sum().sum()
#     r, k = conf_matrix.shape
#     return np.sqrt(chi2 / (n * (min(r, k)-1)))
# print("Cramér’s V:", cramers_v(contingency))
# sns.countplot(data=train, x='Marital Status', hue='Number_of_Dependents_bin')
# plt.title(':포장: Countplot of Marital Status by Dependents Bin')
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()
# # 4) Mutual Information 분석 (혼합형)
# df_enc = train.copy()

# # ––––– 범주형 변수 인코딩 –––––
# for col in df_enc.select_dtypes(include=['object', 'category']).columns:
#     df_enc[col] = LabelEncoder().fit_transform(df_enc[col].astype(str))

# # ––––– X, y 준비 –––––
# y = df_enc['Premium Amount']
# X = df_enc.drop(columns=['Premium Amount'])

# # ––––– NaN 있는 행 제거 –––––
# mask = X.notnull().all(axis=1)
# X_clean = X[mask]
# y_clean = y[mask]

# # ––––– Mutual Information 계산 –––––
# mi = mutual_info_regression(X_clean, y_clean, discrete_features='auto')
# mi_df = pd.DataFrame({'feature': X_clean.columns, 'mutual_info': mi})
# mi_df = mi_df.sort_values(by='mutual_info', ascending=False)

# # ––––– 결과 시각화 –––––
# plt.figure(figsize=(12, 6))
# sns.barplot(x='feature', y='mutual_info', data=mi_df)
# plt.title(':뇌: Mutual Information with Target')
# plt.xticks(rotation=90)
# plt.tight_layout()
# plt.show()